In [1]:
import logging
import datetime

import matplotlib.pyplot as plt

import numpy as np
import torch
import torch.nn as nn
from torch.nn.utils import parameters_to_vector
import torch.optim as optim
from torchinfo import summary

import config
import modules.dataloaders as data_loaders
import modules.utils as utils
import modules.model_mobilenetv2_det_brevitas as model_det
import modules.loss as loss_module
import modules.metrics as metrics
import modules.train_epoch as train_epoch
import modules.val_epoch as val_epoch

from brevitas.export import export_onnx_qcdq

/opt/conda/envs/pytorch_brevitas/lib/python3.10/site-packages/onnxscript/converter.py:820: FutureWarning: 'onnxscript.values.Op.param_schemas' is deprecated in version 0.1 and will be removed in the future. Please use '.op_signature' instead.
  param_schemas = callee.param_schemas()
/opt/conda/envs/pytorch_brevitas/lib/python3.10/site-packages/onnxscript/converter.py:820: FutureWarning: 'onnxscript.values.OnnxFunction.param_schemas' is deprecated in version 0.1 and will be removed in the future. Please use '.op_signature' instead.
  param_schemas = callee.param_schemas()


# Logger

In [2]:
log_path = config.LOGS_FOLDER

logger = logging.getLogger("GonLogger")
logger.propagate = False
logger.setLevel(logging.INFO)
file_handler = logging.FileHandler(log_path + 'logfile.log')
formatter = logging.Formatter('%(message)s')
file_handler.setFormatter(formatter)

# add file handler to logger
logger.addHandler(file_handler)

logger.info(f'{config.MODEL} Detector.\n' + 
            '\tNo Sigmoid, No Softmax. Permute as a Layer.\n' +
            '\tAlbumentations: area changed to 8*8, to keep little boxes.\n' +
            '\tAlbumentations: added ShiftScaleRotate p=0.1.\n' +
            '\tDFire and FASDD UAV and CV.\n' +
            '\tFASDD: train and val datasets to train and test dataset to validate.\n' +
            '\tBrevitas Quantization.\n')

# Hyperparameters Log

In [3]:
''' ============================
    Print Config Values
============================ '''
print('\nDatasets Length')
print(f'\tTrain: {"Full" if config.DS_LEN == None else config.DS_LEN}')
print(f'\tVal: {"Full" if config.VAL_DS_LEN == None else config.VAL_DS_LEN}')
print(f'Device: {config.DEVICE}')
print('Optimizer:')
print(f'\tLearning Rate: {config.LEARNING_RATE}')
print(f'\tGradients Clip Norm: {config.GRADIENTS_CLIP_NORM}')
print(f'\tWeight Decay: {config.WEIGHT_DECAY}')
print('Scheduler:')
print(f'\tScheduler factor: {config.FACTOR}')
print(f'\tScheduler patience: {config.PATIENCE}')
print(f'\tScheduler threshold: {config.THRES}')
print(f'\tScheduler min learning rate: {config.MIN_LR}')
print(f'Batch Size: {config.BATCH_SIZE}')
print(f'Num Workers: {config.NUM_WORKERS}')
print(f'Pin Memory: {config.PIN_MEMORY}')
print(f'Epochs: {config.EPOCHS}')
print('\nIMG DIMS:')
print(f'\tWidth: {config.IMG_W}\n\tHeight: {config.IMG_H}')
print('\nGrid, Bounding Boxes, Classes, Max Obj and Thresholds:')
print(f'\tGrid: {config.S}')
print(f'\tNumber of Bounding Boxes per Cell: {config.B}')
print(f'\tNumber of Classes: {config.C}')
print(f'\tMaximum Number of Objects per Image: {config.MAX_OBJ}')
print(f'\tIOU Threshold: {config.IOU_THRESHOLD}')
print(f'\tScore Threshold: {config.SCORE_THRESHOLD}')
print('\nBrevitas Config:')
print(f'\tFixed Point: {config.FIXED_POINT}')
print(f'\tWeights Bit Width: {config.WEIGHTS_BIT_WIDTH}')
print(f'\tBig Layers Weights Bit Width: {config.BIG_LAYERS_WEIGHTS_BIT_WIDTH}')
print(f'\tHead Weights Bit Width: {config.HEAD_WEIGHTS_BIT_WIDTH}')
print(f'\tBias Bit Width: {config.BIAS_BIT_WIDTH}')
print(f'\tActivations Bit Width: {config.ACTIVATIONS_BIT_WIDTH}')

logger.info('\nDatasets Length')
logger.info(f'\tTrain: {"Full" if config.DS_LEN == None else config.DS_LEN}')
logger.info(f'\tVal: {"Full" if config.VAL_DS_LEN == None else config.VAL_DS_LEN}')
logger.info(f'\nDevice: {config.DEVICE}')
logger.info('Optimizer:')
logger.info(f'\tLearning Rate: {config.LEARNING_RATE}')
logger.info(f'\tGradients Clip Norm: {config.GRADIENTS_CLIP_NORM}')
logger.info(f'\tWeight Decay: {config.WEIGHT_DECAY}')
logger.info('Scheduler:')
logger.info(f'\tScheduler factor: {config.FACTOR}')
logger.info(f'\tScheduler patience: {config.PATIENCE}')
logger.info(f'\tScheduler threshold: {config.THRES}')
logger.info(f'\tScheduler min learning rate: {config.MIN_LR}')
logger.info(f'\nBatch Size: {config.BATCH_SIZE}')
logger.info(f'Num Workers: {config.NUM_WORKERS}')
logger.info(f'Pin Memory: {config.PIN_MEMORY}')
logger.info(f'Epochs: {config.EPOCHS}')
logger.info('\nIMG DIMS:')
logger.info(f'\tWidth: {config.IMG_W}\n\tHeight: {config.IMG_H}')
logger.info('\nGrid, Bounding Boxes, Classes and Thresholds:')
logger.info(f'\tGrid: {config.S}')
logger.info(f'\tNumber of Bounding Boxes per Cell: {config.B}')
logger.info(f'\tNumber of Classes: {config.C}')
logger.info(f'\tMaximum Number of Objects per Image: {config.MAX_OBJ}')
logger.info(f'\tIOU Threshold: {config.IOU_THRESHOLD}')
logger.info(f'\tScore Threshold: {config.SCORE_THRESHOLD}')
logger.info('\nBrevitas Config:')
logger.info(f'\tFixed Point: {config.FIXED_POINT}')
logger.info(f'\tWeights Bit Width: {config.WEIGHTS_BIT_WIDTH}')
logger.info(f'\tBig Layers Weights Bit Width: {config.BIG_LAYERS_WEIGHTS_BIT_WIDTH}')
logger.info(f'\tHead Weights Bit Width: {config.HEAD_WEIGHTS_BIT_WIDTH}')
logger.info(f'\tBias Bit Width: {config.BIAS_BIT_WIDTH}')
logger.info(f'\tActivations Bit Width: {config.ACTIVATIONS_BIT_WIDTH}')


Datasets Length
	Train: 200
	Val: 200
Device: cuda
Optimizer:
	Learning Rate: 0.001
	Gradients Clip Norm: 10
	Weight Decay: 0.001
Scheduler:
	Scheduler factor: 0.8
	Scheduler patience: 3
	Scheduler threshold: 0.01
	Scheduler min learning rate: 1e-06
Batch Size: 64
Num Workers: 8
Pin Memory: True
Epochs: 5

IMG DIMS:
	Width: 224
	Height: 224

Grid, Bounding Boxes, Classes, Max Obj and Thresholds:
	Grid: 7
	Number of Bounding Boxes per Cell: 2
	Number of Classes: 2
	Maximum Number of Objects per Image: 10
	IOU Threshold: 0.5
	Score Threshold: 0.2

Brevitas Config:
	Fixed Point: True
	Weights Bit Width: 4
	Big Layers Weights Bit Width: 4
	Head Weights Bit Width: 8
	Bias Bit Width: 4
	Activations Bit Width: 4


# Datasets and Dataloaders

In [4]:
train_loader = data_loaders.get_train_loader()
val_loader = data_loaders.get_val_loader()


TRAIN DFIRE dataset
DFire Removed wrong images: 0
DFire Removed due to overlapping: 16
DFire Removed due to more than 10: 1

Train DFire dataset len: 183

TRAIN FASDD UAV dataset
FASDD Removed wrong images: 0
FASDD Removed due to overlapping: 24
FASDD Removed due to more than 10: 3

Train FASDD UAV dataset len: 173

VAL FASDD UAV dataset
FASDD Removed wrong images: 0
FASDD Removed due to overlapping: 17
FASDD Removed due to more than 10: 7

Val FASDD UAV dataset len: 176

TRAIN FASDD CV dataset
FASDD Removed wrong images: 0
FASDD Removed due to overlapping: 4
FASDD Removed due to more than 10: 0

Train FASDD CV dataset len: 196

VAL FASDD CV dataset
FASDD Removed wrong images: 0
FASDD Removed due to overlapping: 8
FASDD Removed due to more than 10: 3

Val FASDD CV dataset len: 189

Concatenate Train DFire and Train FASDD UAV datasets
Train dataset len: 356
Concatenate with Val FASDD UAV dataset
Train dataset len: 532
Concatenate with Train FASDD CV dataset
Train dataset len: 728
Conca

### Plot Some Train Pictures

In [5]:
for batch_idx, (img, label) in enumerate(train_loader):
       
    if batch_idx == 0:
        print(f'Batch size equal to img.shape[0] = {img.shape[0]}')
        print(f'Batch images shape = {img.shape}')
        plt.subplots(4, 5, figsize=(10,8))
        for i in range(20):
            pic = utils.plot_dataset_img(img[i], label[i], grid=True)
            plt.subplot(4, 5, i+1)
            plt.imshow(pic)
        plt.tight_layout()
        plt.savefig(config.RUN_FOLDER + 'train_pictures.png')
        plt.close()
        break

Batch size equal to img.shape[0] = 64
Batch images shape = torch.Size([64, 3, 224, 224])


### Plot Some Val Pictures

In [6]:
for batch_idx, (img, label) in enumerate(val_loader):
       
    #if batch_idx == 33:
    if batch_idx == 3:
        print(f'Batch size equal to img.shape[0] = {img.shape[0]}')
        print(f'Batch images shape = {img.shape}')
        plt.subplots(4, 5, figsize=(10,8))
        for i in range(20):
            pic = utils.plot_dataset_img(img[i], label[i], grid=True)
            plt.subplot(4, 5, i+1)
            plt.imshow(pic)
        plt.tight_layout()
        plt.savefig(config.RUN_FOLDER + 'val_pictures.png')
        plt.close()
        break

Batch size equal to img.shape[0] = 64
Batch images shape = torch.Size([64, 3, 224, 224])


# Loss Setup

In [7]:
if config.LOSS_FN == "YOLOV1_LOSS":
    print(f'Loss Function: YOLOV1_LOSS')
    logger.info(f'\nLoss Function: YOLOV1_LOSS')
    loss_fn = loss_module.YoloLoss_2BBox()
    print(f'Lambda for L1 regularization: {config.LAMBDA_L1_LOSS}')
    logger.info(f'Lambda for L1 regularization: {config.LAMBDA_L1_LOSS}')
else:
    print("Wrong loss function")
    logger.info("Wrong loss function")
    raise SystemExit("Wrong loss function")

Loss Function: YOLOV1_LOSS
Lambda for L1 regularization: 0


# Model Setup

In [8]:
model = model_det.MobileNetV2_DET_BREVITAS().to(config.DEVICE)

optimizer = optim.Adam(model.parameters(), 
                       lr=config.LEARNING_RATE, 
                       weight_decay=config.WEIGHT_DECAY)

scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, 
                                                 mode='min',
                                                 factor=config.FACTOR, 
                                                 patience=config.PATIENCE, 
                                                 threshold=config.THRES, 
                                                 threshold_mode='abs',
                                                 min_lr=config.MIN_LR)

# MODEL PARAMETERS
n_trainable = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f'\nTrainable parameters = {n_trainable}')
logger.info(f'\nTrainable parameters = {n_trainable}')

n_params = parameters_to_vector(model.parameters()).numel()
print(f'Total parameters = {n_params}\n')
logger.info(f'Total parameters = {n_params}')


Trainable parameters = 307761
Total parameters = 307761



/opt/conda/envs/pytorch_brevitas/lib/python3.10/site-packages/brevitas/nn/mixin/base.py:77: UserWarning: Keyword arguments are being passed but they not being used.
  warn('Keyword arguments are being passed but they not being used.')


### Check Model Shape

In [9]:
in_rand_np = np.random.rand(4, 3, config.IMG_H, config.IMG_W)
in_rand = torch.tensor(in_rand_np, dtype=torch.float32, device=config.DEVICE)
out_test = model(in_rand);

print(f'Input shape is {in_rand.shape}')
print(f'Model shape is {out_test.shape}')
# print(f'Model Arquitecture\n{model}')
logger.info(f'\nInput shape is {in_rand.shape}')
logger.info(f'Model shape is {out_test.shape}\n')
# logger.info(f'Model Arquitecture\n{model}')

/opt/conda/envs/pytorch_brevitas/lib/python3.10/site-packages/torch/_tensor.py:1362: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at /opt/conda/conda-bld/pytorch_1702400410390/work/c10/core/TensorImpl.h:1900.)
  return super().rename(names)


Input shape is torch.Size([4, 3, 224, 224])
Model shape is torch.Size([4, 12, 7, 7])


### Torch Summary

In [10]:
model.eval()

print(summary(model, input_size=(1, 3, config.IMG_H, config.IMG_W)))
logger.info("\nORIGINAL Model Summary")
logger.info(summary(model, input_size=(1, 3, config.IMG_H, config.IMG_W)))

Layer (type:depth-idx)                                                           Output Shape              Param #
MobileNetV2_DET_BREVITAS                                                         [1, 12, 7, 7]             --
├─Sequential: 1-1                                                                [1, 128, 7, 7]            297,901
│    └─QuantIdentity: 2-1                                                        [1, 3, 224, 224]          --
│    │    └─ActQuantProxyFromInjector: 3-1                                       [1, 3, 224, 224]          --
│    │    └─ActQuantProxyFromInjector: 3-2                                       [1, 3, 224, 224]          1
├─Sequential: 1-110                                                              --                        (recursive)
│    └─Sequential: 2-141                                                         --                        (recursive)
│    │    └─QuantRelu6: 3-157                                                    --          

# Loss and Metrics Loggers and Plotters

In [11]:
train_losses_logger = utils.LogLosses()
train_metrics_logger = utils.LogMetrics()
lr_logger = utils.LogLR(log_path=config.PLOTS_FOLDER)

val_losses_logger = utils.LogLosses()
val_metrics_logger = utils.LogMetrics()

loss_plotter = utils.PlotMetrics(log_path=config.PLOTS_FOLDER, model_name=config.MODEL, loss_or_metric='Loss')
metrics_plotter = utils.PlotMetrics(log_path=config.PLOTS_FOLDER, model_name=config.MODEL, loss_or_metric='Metric')

# Train Loop Function

In [12]:
def train_loop(model, start_epoch=0, epochs_to_train=config.EPOCHS):

    ''' ==============================================================
                                TRAINING LOOP
    ============================================================== '''
    start = datetime.datetime.now()
    start_time = start.strftime("%H:%M:%S")
    print(f'\n***Start Training: {start_time}\n')
    logger.info(f'\n***Start Training: {start_time}\n')
    
    # Start with infinite validation loss
    best_valid_loss = np.inf
    best_mAP = torch.tensor(0., dtype=torch.float32)

    epochs_loss_plot = []
    epochs_metric_plot = []
    if start_epoch != 0:
        # Losses Update
        current_losses = train_losses_logger.get_losses()
        current_total_loss = current_losses['Total']
        current_total_loss_epochs = len(current_total_loss)
        epochs_loss_plot.extend(range(current_total_loss_epochs))  
        # Metrics Update
        current_metrics = train_metrics_logger.get_metrics()
        current_total_mAP = current_metrics['mAP']['mAP']
        current_total_mAP_epochs = len(current_total_mAP)
        epochs_metric_plot.extend(range(current_total_mAP_epochs)) 
        for i, e in enumerate(epochs_metric_plot):
            epochs_metric_plot[i] = 5*epochs_metric_plot[i] + 4  
    
    end_epoch = start_epoch + epochs_to_train
        
    for epoch in range(start_epoch, end_epoch):

        print(f'\n=== EPOCH {epoch}/{end_epoch-1} ===')
        logger.info(f'\n=== EPOCH {epoch}/{end_epoch-1} ===')
        
        #====================== TRAINING ========================#
        current_lr = train_epoch.get_lr(optimizer=optimizer)
        logger.info(f'Learning Rate = {current_lr}\n')
        lr_logger.log_lr(current_lr)
            
        calculate_mAP = False
        if ( (epoch+1) % 5 ) == 0:
            calculate_mAP = True
            epochs_metric_plot.append(epoch)

        train_losses, train_metrics = train_epoch.train_fn(
            loader=train_loader, 
            model=model, 
            optimizer=optimizer, 
            loss_fn=loss_fn,
            loss_l1_lambda=config.LAMBDA_L1_LOSS,
            metric=metrics.map_metric,
            device=config.DEVICE,
            calculate_mAP=calculate_mAP)
        
        train_losses_logger.update_losses(train_losses)
        if calculate_mAP == True:
            train_metrics_logger.update_metrics(train_metrics)
                
        logger.info(utils.print_metrics_to_logger("TRAIN STATS", train_losses, train_metrics, mAP_available=calculate_mAP))
        
        #===================== VALIDATING =======================#
        with torch.no_grad():
            val_losses, val_metrics = val_epoch.eval_fn(
                loader=val_loader, 
                model=model,                         
                loss_fn=loss_fn,
                metric=metrics.map_metric,
                device=config.DEVICE,
                calculate_mAP=calculate_mAP)
            
            scheduler.step(val_losses['Total'])
            
            val_losses_logger.update_losses(val_losses)
            if calculate_mAP == True:
                val_metrics_logger.update_metrics(val_metrics)

            logger.info(utils.print_metrics_to_logger("VAL STATS", val_losses, val_metrics, mAP_available=calculate_mAP))
            
        epochs_loss_plot.append(epoch)

        loss_plotter.plot_all_metrics(
            train_losses_logger.get_losses(),
            val_losses_logger.get_losses(),
            epochs_loss_plot)

        if calculate_mAP == True:
            metrics_plotter.plot_all_metrics(
                train_metrics_logger.get_metrics(),
                val_metrics_logger.get_metrics(),
                epochs_metric_plot)

        lr_logger.plot_lr(epochs_loss_plot)
        
        #======================= SAVING =========================#
        if ( (epoch+1) % 5 ) == 0:
            save_name = config.WEIGHTS_FOLDER + config.MODEL + '_detector__5epoch.pt'
            utils.save_checkpoint(epoch, model, optimizer, scheduler, save_name) 
            
        if best_valid_loss > val_losses['Total']:
            best_valid_loss = val_losses['Total']
            print(f"\nSaving model with new best validation loss: {best_valid_loss:.3f}")
            logger.info(f"Saving model with new best validation loss: {best_valid_loss:.3f}")
            save_name = config.WEIGHTS_FOLDER + config.MODEL + '_detector__' + 'best_loss'  + '.pt'
            utils.save_checkpoint(epoch, model, optimizer, scheduler, save_name) 
            save_onnx = config.ONNX_FOLDER + config.MODEL + '_detector__' + 'best_loss'
            utils.export_onnx(model, (1, config.NUM_CHANNELS, config.IMG_H, config.IMG_W), save_onnx, config.DEVICE)


        # Save model if mAP increases
        if calculate_mAP == True:
            if ( best_mAP < val_metrics['mAP'] ) :
                best_mAP = val_metrics['mAP']
                print(f"\nSaving model with new best mAP: {best_mAP:.4f}")
                logger.info(f"Saving model with new best mAP: {best_mAP:.4f}")
                save_precision_name = f'best_mAP={best_mAP:.4f}__epoch={epoch}'
                save_name = config.WEIGHTS_FOLDER + config.MODEL + '_detector__' + save_precision_name + '.pt'
                utils.save_checkpoint(epoch, model, optimizer, scheduler, save_name)  
                save_onnx = config.ONNX_FOLDER + config.MODEL + '_detector__' + 'best_loss'
                utils.export_onnx(model, (1, config.NUM_CHANNELS, config.IMG_H, config.IMG_W), save_onnx, config.DEVICE)
        
    logger.info('Saving last model')   
    torch.save(model.state_dict(), config.WEIGHTS_FOLDER + 'last_' + config.MODEL + '_detector.pt') 
    
    #======================= FINISH =========================#
    end = datetime.datetime.now()
    end_time = end.strftime("%H:%M:%S")
    print(f'\n***Script finished: {end_time}\n')  
    print(f'Time elapsed: {end-start}')
    logger.info(f'\n***Script finished: {end_time}\n')  
    logger.info(f'Time elapsed: {end-start}')
    
    return model

# Main Execute

In [13]:
print("Starting script\n")
logger.info("Starting script\n")
    
model = train_loop(model)

Starting script


***Start Training: 01:14:46


=== EPOCH 0/4 ===
Learning Rate = 0.001



Training: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 14/14 [00:05<00:00,  2.36it/s]


Total Loss  |Box Loss    |Conf Loss   |No Obj Loss |Class Loss  
------------ ------------ ------------ ------------ ------------
698.992     |383.365     |54.276      |178.257     |83.094      


Validating: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 8/8 [00:02<00:00,  3.50it/s]


Total Loss  |Box Loss    |Conf Loss   |No Obj Loss |Class Loss  
------------ ------------ ------------ ------------ ------------
560.698     |412.493     |49.632      |17.624      |80.948      

Saving model with new best validation loss: 560.698
Model exported to ONNX: experiments/test_10_307K_128_ds/onnx/MOBILENETV2_DET_detector__best_loss

=== EPOCH 1/4 ===
Learning Rate = 0.001



...

Total Loss  |Box Loss    |Conf Loss   |No Obj Loss |Class Loss  
------------ ------------ ------------ ------------ ------------
220.685     |114.958     |49.811      |14.669      |41.247      

Saving model with new best validation loss: 220.685
Model exported to ONNX: experiments/test_10_307K_128_ds/onnx/MOBILENETV2_DET_detector__best_loss

=== EPOCH 4/4 ===
Learning Rate = 0.001



Training: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 14/14 [00:06<00:00,  2.13it/s]


Total Loss  |Box Loss    |Conf Loss   |No Obj Loss |Class Loss  
------------ ------------ ------------ ------------ ------------
202.330     |97.614      |54.057      |13.196      |37.463      
Train mAP = 0.0005


Validating: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 8/8 [00:02<00:00,  3.33it/s]


Total Loss  |Box Loss    |Conf Loss   |No Obj Loss |Class Loss  
------------ ------------ ------------ ------------ ------------
193.239     |97.368      |49.156      |11.930      |34.785      
Val mAP = 0.0001

Saving model with new best validation loss: 193.239
Model exported to ONNX: experiments/test_10_307K_128_ds/onnx/MOBILENETV2_DET_detector__best_loss

Saving model with new best mAP: 0.0001
Model exported to ONNX: experiments/test_10_307K_128_ds/onnx/MOBILENETV2_DET_detector__best_loss

***Script finished: 01:15:39

Time elapsed: 0:00:53.010233
